Домашнее задание по АОТ

#это переделка первого ДЗ с учётом биграмм, читать где-то с середины, где начинается новое

In [87]:
! pip install requests
! pip install nltk
! pip install pymorphy2

In [88]:
import requests
import re
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Я взяла сайт LiveLib, отзывы на книгу "девушка в поезде".

Напишем штуку, которая их собирает

In [89]:
def html_collect():
  result = requests.get('https://www.livelib.ru/book/1001526145/reviews-devushka-v-poezde-pola-hokins')
  result.encoding = 'utf-8'
  prob = result.text

  html = [prob]
  for i in range(2, 20):
    link = 'https://www.livelib.ru/book/1001526145/reviews/~'+str(i)+'#reviews'
    result = requests.get(link)
    result.encoding = 'utf-8'
    html.append(result.text)
  return(html)

Один раз соберем их чтобы лишний раз потом не дергать

In [90]:
temp = html_collect()

Эта функция ищет в коде страницы то, что нам нужно: сами отзывы и оценки

In [91]:
def rev_search(html):
  revs = []
  for i in html:
    rev_all = re.compile('<span class="lenta-card__mymark">(.*?)</span>.*?<div id="lenta-card__text-review-escaped">.*?<p><p>(.*?)</p></p>', flags = re.DOTALL)
    revs.append(rev_all.findall(i))
  return(revs)

Эта функция аккуратно собирает их в список

In [ ]:
def make_rev_list(revs):
  revlist = []
  for i in revs[0]:
    revlist.append(i)
  for i in range(1, len(revs)):
    for j in range(1, len(revs[i])):
      revlist.append(revs[i][j])
  return(revlist)

Эта функция чистит содержимое от тегов и переносов строки

In [ ]:
def rev_clean(revs):
  clean_revs=[]
  for i in revs:
    x = re.sub(r'\n', '', i[0])
    x = re.sub(' ', '', x)
    y = re.sub('<.*?>', '', i[1])
    y = re.sub('\n', ' ', y)
    y = re.sub('  ', ' ', y)
    clean_revs.append([y, x])
  return(clean_revs)

Эта функция считает, что оценка 4 или выше это положительный отзыв, 2.5 или ниже --- отрицательный

In [ ]:
def rating(x):
  if float(x) >= 4:
    r = 'good'
  elif float(x) <= 2.5:
    r = 'bad'
  else:
    r = ''
  return(r)

Эта функция находит создает списки положительных и отрицательных рецензий

In [ ]:
def find_polar(revs):
  good = [i for i in revs if rating(i[1]) == 'good']
  bad = [i for i in revs if rating(i[1]) == 'bad']
  return(good, bad)

Эта функция делает нам списки из 30 положительных и 30 отрицательных отзывов как надо в задании

In [ ]:
def make_data(good, bad):
  good1 = good[:31]
  del good1[24]
  bad1 = bad[:30]
  test1 = good[35:40] + bad[31:36]
  return good1, bad1, test1

Эта функция разбивает отзывы определенной полярности на слова, приводит их к нижнему регистру, чистит и лемматизирует

In [ ]:
def polar_words(polar):
  pwords = []
  for i in polar:
    pw = word_tokenize(i[0])
    pwords.extend(pw)
  smallpwords = [w.lower() for w in pwords if w.isalpha()]
  polarwords = [morph.parse(w)[0].normal_form for w in smallpwords] 
  return polarwords

Эта функция чистит списки от слов, которые есть в списке другой полярности (то есть оставляет только слова из положительных или отрицательных отзывов)

In [ ]:
def only_words(polar1, polar2):
  return [w for w in polar1 if w not in polar2]

Эта функция убирает слова, которые встречались меньше 3 раз

In [ ]:
def only_freq(polar):
  pcounter = Counter(polar)
  return [w for w, val in pcounter.items() if val > 2]

Эта функция считает полярность отзыва. Она считает сколько в нем положительных слов к общему числу положительных слов и аналогично для отрицательных. Так логичнее, потому что отрицательных слов получилось больше, чем положительных.

In [ ]:
def polar(review, good, bad):
  rg = 0
  rb = 0
  wrd = word_tokenize(review)
  wrd1 = [w.lower() for w in wrd if w.isalpha()]
  lms = [morph.parse(w)[0].normal_form for w in wrd1]
  for i in lms:
    if i in good:
      rg += 1
    elif i in bad:
      rb += 1
  if rg/len(good)>rb/len(bad):
    p = 'good'
  else:
    p = 'bad'
  return(p)

Эта функция считает точность для тестового набора отзывов.

In [ ]:
def accuracy(test, good, bad):
  acc = 0
  for i in test:
    if polar(i[0], good, bad) == rating(i[1]):
      acc += 1
  return(acc/len(test))

Здесь мы применяем все функции по очереди чтоб получить результат.

In [ ]:
reviews = rev_clean(make_rev_list(rev_search(temp))) # все отзывы
good0, bad0 = find_polar(reviews) # разделили на хорошие и плохие
goodrevs, badrevs, test = make_data(good0, bad0) # создали данные
goodwords = polar_words(goodrevs) # нашли слова из хороших отзывов
badwords = polar_words(badrevs) # нашли слова из плохих отзывов
freqgood = only_freq(only_words(goodwords, badwords)) # взяли только частотные и только из хороших
freqbad = only_freq(only_words(badwords, goodwords)) # взяли только частотные и только из плохих
accuracy(test, freqgood, freqbad) # посчитали точность

0.6

Ура, неплохая точность, 6/10! Но вряд ли точности, подсчитанной на 10 отзывах можно доверять :)

Как улучшить программу?

Во-первых, как всегда, взять гораааздо больше рецензий. После этого игра с весами (насколько нечастотные слова можно оставить) имела бы больше смысла (я играла, в моем случае >2 оптимальный вариант).

Во-вторых можно попробовать находить частотные для только хороших или плохих отзывов биграммы. Не факт что это поможет, например, в плохих отзывах будет биграмма "ужасная книга", но слово "ужасный" скорее всего и так относится к плохим словам. 

#читать отсюда

Будем улучшать программу добавлением биграмм.

In [ ]:
!pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-tv_ulbnc
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-tv_ulbnc
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9922 sha256=cde8bae6d4a612f7dd96ce436bb0c518ec5c45dfe1a186fa7e26f09c6af34f03
  Stored in directory: /tmp/pip-ephem-wheel-cache-21ojnsqo/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [ ]:
from pymystem3 import Mystem
m2 = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-18 18:09:13--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.242, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-18 18:09:14--  http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)... 5.45.220.16, 2a02:6b8:0:2002::17
Connecting to cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)|5.45.220.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [ ]:
from nltk.tokenize import sent_tokenize

Лемматизируем и попостэгаем текст

In [ ]:
def myst(rutext):
  ana = m2.analyze(rutext)
  myswordpos = []
  for word in ana:
    if 'analysis' in word:
      if word['analysis'] == []:
        lex = word['text']
        pos = ''
      else:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        lex = word['analysis'][0]['lex']
      myswordpos.append([word['text'], pos, lex])
  return(myswordpos)

Функция, которая находит в тексте биграммы типа прил+сущ

In [ ]:
def adj_noun (sent):
  chunks = []
  words = myst(sent)
  #print(words)
  for i in range(len(words)-1):
    if words[i][1] == 'A' and words[i+1][1] == 'S':
      chunks.append(words[i][2]+' '+words[i+1][2])
  return(chunks)

Функция, которая из данного списка рецензий сделает два списка: список подходящих биграмм и список подходящих биграмм, которые встречаются больше одного раза.

In [ ]:
def AN_get(revs):
  an = []
  anmore = []
  for i in revs:
    sents = sent_tokenize(i[0])
    for j in sents:
      if adj_noun(j) != []:
        for k in adj_noun(j):
          if k not in an:
            an.append(k)
          elif k not in anmore:
            anmore.append(k)
          #print(k)
  return(an, anmore)

Применим эти функции и получим списки подходящих биграмм, которые встречаются только в отзывах своей полярности.

In [ ]:
angood, goodmore = AN_get(goodrevs)
anbad, badmore = AN_get(badrevs)
angoodonly = []
anbadonly = []
for i in goodmore:
  if i not in anbad:
    angoodonly.append(i)
for i in badmore:
  if i not in angood:
    anbadonly.append(i)
print(angoodonly)
print(anbadonly)

['разрушительный подозрение', 'сюжетный поворот', 'разный человек']
['разный женщина', 'литературный язык']


Сделаем всё то же самое для биграмм типа (любое слово)+книга

In [ ]:
def any_kniga (sent):
  chunks = []
  words = myst(sent)
  #print(words)
  for i in range(len(words)-1):
    if words[i+1][2] == 'книга':
      chunks.append(words[i][2]+' '+words[i+1][2])
  return(chunks)

In [ ]:
def any_kniga_get(revs):
  an = []
  anmore = []
  for i in revs:
    sents = sent_tokenize(i[0])
    for j in sents:
      if any_kniga(j) != []:
        for k in any_kniga(j):
          if k not in an:
            an.append(k)
          elif k not in anmore:
            anmore.append(k)
          #print(k)
  return(an, anmore)

In [ ]:
akgood, goodakmore = any_kniga_get(goodrevs)
akbad, badakmore = any_kniga_get(badrevs)
akgoodonly = []
akbadonly = []
for i in goodakmore:
  if i not in akbad:
    akgoodonly.append(i)
for i in badakmore:
  if i not in akgood:
    akbadonly.append(i)
print(akgoodonly)
print(akbadonly)

['сюжет книга', 'что книга']
['конец книга', 'середина книга', 'плюс книга']


Сделаем то же для биграмм типа (любое слово)+сюжет

In [ ]:
def any_plot (sent):
  chunks = []
  words = myst(sent)
  #print(words)
  for i in range(len(words)-1):
    if words[i+1][2] == 'сюжет':
      chunks.append(words[i][2]+' '+words[i+1][2])
  return(chunks)

In [ ]:
def any_plot_get(revs):
  an = []
  anmore = []
  for i in revs:
    sents = sent_tokenize(i[0])
    for j in sents:
      if any_plot(j) != []:
        for k in any_plot(j):
          if k not in an:
            an.append(k)
          elif k not in anmore:
            anmore.append(k)
          #print(k)
  return(an, anmore)

In [ ]:
apgood, goodapmore = any_plot_get(goodrevs)
apbad, badapmore = any_plot_get(badrevs)
apgoodonly = []
apbadonly = []
for i in goodapmore:
  if i not in badapmore:
    apgoodonly.append(i)
for i in badapmore:
  if i not in goodapmore:
    apbadonly.append(i)
print(apgoodonly)
print(apbadonly)

['и сюжет', 'в сюжет']
[]


Соединим списки полярных биграмм.

In [ ]:
badbi = angoodonly + apbadonly + akbadonly
goodbi = anbadonly + apgoodonly + akgoodonly
print(badbi)
print(goodbi)

['разрушительный подозрение', 'сюжетный поворот', 'разный человек', 'конец книга', 'середина книга', 'плюс книга']
['разный женщина', 'литературный язык', 'и сюжет', 'в сюжет', 'сюжет книга', 'что книга']


Напишем новую функцию (точнее отредактируем старую), которая будет считать полярность отзыва с учетом биграмм.

In [ ]:
def polar2(review, good, bad, goodbi, badbi):
  rg = 0
  rb = 0
  wrd = word_tokenize(review)
  wrd1 = [w.lower() for w in wrd if w.isalpha()]
  lms = [morph.parse(w)[0].normal_form for w in wrd1]
  for i in lms:
    if i in good:
      rg += 1
    elif i in bad:
      rb += 1
  sents = sent_tokenize(review)
  for i in sents:
    for j in (adj_noun(i)+any_kniga(i)+any_plot(i)):
      if j in goodbi:
        rg += 1
        print(j)
      elif j in badbi:
        rb += 1
        print(j)
  if rg/(len(good)+len(goodbi))>rb/(len(bad)+len(badbi)):
    p = 'good'
  else:
    p = 'bad'
  return(p)

Напишем новую функцию, кторая будет считать точность используя новую функцию polar2

In [ ]:
def accuracy2(test, good, bad):
  acc = 0
  for i in test:
    if polar2(i[0], good, bad, goodbi, badbi) == rating(i[1]):
      acc += 1
  return(acc/len(test))

Посчитаем новую точность

In [ ]:
accuracy2(test, freqgood, freqbad)

0.6

Тадам, она такая же. Кто бы сомневался))))))))))

Но наверняка всё было не зря и при большем кол-ве отзывов точность была бы выше.

Почему я взяла эти биграммы? Биграмма adj+noun вообще популярная, там вполне могли бы встретиться какие-то повторения. Биграммы сос словами книга и сюжет тоже вполне логичные, потому что у меня ревью на книгу, а в книге важно что? сюжет...